# FINM 35000 Problem Set 3: Equity Valuation Stress Testing

<span style="color:blue">Aman Krishna </span> <br>
<br>
<span style="color:#406A5F">Tim Taylor </span> <br>
<br>
<span style="color:purple">Yazmin Ramirez Delgado </span>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math as m 
import scipy.stats as stats
import datetime as dt
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
from scipy.stats import norm
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore")

# 1. Replication of Cosemans and Frehen (2021) (100 points)

Note: for questions 2-3, it is possible you will not obtain the exact numbers in the paper, which is okay as long as you are able to describe the ways in which you might have deviated from the authors (in question 4).

### 1. 
In your own words, describe what the authors mean by “salience theory” and how it affects investor’s portfolio choice decisions.

<span style="color:purple">"Salience theory," as discussed the paper, refers to the idea that investors tend to give disproportionate attention and importance to the most prominent or striking features of an investment, particularly past returns. This theory is grounded in the broader understanding of how cognitive biases (something like behavioural economics) influence decision-making. In the context of stock market investments, salience theory suggests that investors are drawn to stocks that have had notably high or low returns in the past, as these returns are more "salient" or noticeable. Famous stocks like Apple and Tesla come to mind when thinking about this theory from a US Stock market perspective. </span> <br>
<br>

<span style="color:blue"> According to salience theory, investors do not evaluate potential investments in a completely rational or comprehensive manner. That is, investors are unsophisticated in their decision making and, they are more likely to focus on the most memorable or striking aspects of an asset's history, especially its past performance. For example, if a stock has experienced a significant upsurge in value in the recent past, this positive performance becomes a salient feature that attracts investors, leading them to overvalue such stocks. This overvaluation, in turn, means that these stocks are likely to have lower future returns because their current prices are inflated due to high demand based on salient past performance.</span> <br>
<br>

<span style="color:#406A5F"> Going the other way, stocks with notably poor past performance can become undervalued, as investors overlook them due to their salient negative returns. These undervalued stocks, according to the theory, are likely to yield higher future returns as their current lower prices do not reflect their potential value. </span> <br>
<br>

<span style="color:#406A5F"> Overall, we saw a lot of similarity between Fama French's fourth factor (out of 5) - Profitability (RMW - Robust Minus Weak). This factor captures the historical outperformance of profitable companies compared to less profitable ones. It measures the return difference between a portfolio of companies with high profitability and a portfolio of companies with low profitability. </span> <br>
<br>

### 2. 
Following Section 3 of the paper, download the relevant variables from CRSP and Compustat (both available through WRDS). Use this data to replicated Table 2.

Load CRSP Daily, Monthly, and Compustat Fundamentals Data

In [6]:
# Read "C:\Users\Aman\Downloads\Compressed\crsp_us_equity.csv"
crsp_daily = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_us_equity.csv")

In [7]:
# Convert date to datetime format
crsp_daily['date'] = pd.to_datetime(crsp_daily['date'])

# Sort the DataFrame by 'TICKER' and 'date'
crsp_daily = crsp_daily.sort_values(['TICKER', 'date'])

# Remove all rows with missing TICKER or RET
crsp_daily.dropna(subset=['TICKER', 'RET'], inplace=True)


In [8]:
# Create a month and year column like 2005-01
crsp_daily['month'] = crsp_daily['date'].dt.strftime('%Y-%m')

In [9]:
# Use groupby and transform to calculate the number of days in each month
crsp_daily['days_in_month'] = crsp_daily.groupby(['TICKER', 'month'])['RET'].transform('count')

# # Set data and ticker as index
# crsp_daily = crsp_daily.set_index(['date', 'TICKER'])

# # Remove all dates before 2000-01-01
# crsp_daily = crsp_daily[crsp_daily['date'] >= '2005-01-01']

In [10]:
crsp_daily

,PERMNO,date,TICKER,PRC,VOL,RET,month,days_in_month
1709955,10495,1962-07-02,A,41.125,2600.000,0.021739,1962-07,21
1709956,10495,1962-07-03,A,41.375,2100.000,0.006079,1962-07,21
1709957,10495,1962-07-05,A,41.250,3600.000,-0.003021,1962-07,21
1709958,10495,1962-07-06,A,40.500,2600.000,-0.018182,1962-07,21
1709959,10495,1962-07-09,A,40.750,4000.000,0.006173,1962-07,21
...,...,...,...,...,...,...,...,...
85532128,91205,2013-03-11,ZZ,2.200,407000.000,0.000000,2013-03,11
85532129,91205,2013-03-12,ZZ,2.210,159900.000,0.004545,2013-03,11
85532130,91205,2013-03-13,ZZ,2.210,308900.000,0.000000,2013-03,11
85532131,91205,2013-03-14,ZZ,2.210,274900.000,0.000000,2013-03,11


In [11]:
# Read "C:\Users\Aman\Downloads\Compressed\crsp_us_equity_monthly.csv"
crsp_monthly = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_us_equity_monthly.csv")

In [12]:
# Convert 'date' column to datetime
crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])

# Sort the DataFrame by 'TICKER' and 'date' columns
crsp_monthly.sort_values(by=['TICKER', 'date'], inplace=True)

# Remove all rows with missing TICKER
crsp_monthly.dropna(subset=['TICKER'], inplace=True)

# Convert negative PRC values to positive
crsp_monthly['PRC'] = crsp_monthly['PRC'].abs()

# Fill missing PRC values with 0
crsp_monthly['PRC'].fillna(0, inplace=True)

# Shift the indexes by 1 for crsp_monthly so that the PRC, VOL and RET values are for the previous month
crsp_monthly['PRC'] = crsp_monthly.groupby(['TICKER'])['PRC'].shift(1)

# # Remove all dates before 2000-01-01
# crsp_monthly = crsp_monthly[crsp_monthly['date'] >= '2005-01-01']

# Backfill the missing PRC values with next available PRC value
crsp_monthly['PRC'].fillna(method='bfill', inplace=True)

#Drop COMNAM and PERMNO columns
crsp_monthly.drop(columns=['COMNAM'], inplace=True)

# # Set data and ticker as index
# crsp_monthly = crsp_monthly.set_index(['date', 'TICKER'])

In [13]:
crsp_monthly

,PERMNO,date,TICKER,PRC,VOL,RET
80206,10495,1962-07-31,A,40.375,852.000,0.003106
80207,10495,1962-08-31,A,40.375,967.000,0.024768
80208,10495,1962-09-28,A,40.875,1525.000,-0.094801
80209,10495,1962-10-31,A,37.000,1396.000,0.033784
80210,10495,1962-11-30,A,38.250,1895.000,0.117647
...,...,...,...,...,...,...
4083922,91205,2012-11-30,ZZ,2.230,111189.000,-0.026906
4083923,91205,2012-12-31,ZZ,2.170,116706.000,0.000000
4083924,91205,2013-01-31,ZZ,2.170,71494.000,-0.004608
4083925,91205,2013-02-28,ZZ,2.160,97674.000,0.009259


In [14]:
# Merge crsp_daily and crsp_monthly on TICKER, date, PERMNO
crsp = pd.merge(crsp_daily, crsp_monthly, on=['TICKER','PERMNO', 'date'], how='outer')

In [16]:
del crsp_daily, crsp_monthly

In [15]:
crsp

,PERMNO,date,TICKER,PRC_x,VOL_x,RET_x,month,days_in_month,PRC_y,VOL_y,RET_y
0,10495,1962-07-02,A,41.125,2600.000,0.021739,1962-07,21.000,NaN,NaN,NaN
1,10495,1962-07-03,A,41.375,2100.000,0.006079,1962-07,21.000,NaN,NaN,NaN
2,10495,1962-07-05,A,41.250,3600.000,-0.003021,1962-07,21.000,NaN,NaN,NaN
3,10495,1962-07-06,A,40.500,2600.000,-0.018182,1962-07,21.000,NaN,NaN,NaN
4,10495,1962-07-09,A,40.750,4000.000,0.006173,1962-07,21.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
78156370,84188,1986-11-28,ZTXQ,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN
78156371,14327,2015-10-30,ZU,NaN,NaN,NaN,NaN,NaN,17.400,NaN,NaN
78156372,91435,2009-02-27,ZVUE,NaN,NaN,NaN,NaN,NaN,0.021,172009.000,NaN
78156373,85520,2007-03-30,ZVXI,NaN,NaN,NaN,NaN,NaN,12.880,2990.000,NaN


## NOTE FOR USER -  ALL _X VARIABLES ARE FROM CRSP DAILY AND ALL _Y VARIABLES ARE FROM CRSP MONTHLY

#### Only taking >$5 prev month price and >15 days returns in a month

In [18]:
# group by TICKER and backfill the PRC values
crsp['PRC_y_daily'] = crsp.groupby(['TICKER'])['PRC_y'].fillna(method='bfill')
# Filter dataframe where PRC_x is >= 5 and days_in_month >15
crsp = crsp[(crsp['PRC_y_daily'] >= 5) & (crsp['days_in_month'] > 15)]

In [19]:
# Convert all RET_x to float, if not possible, convert to NaN
crsp['RET_x'] = pd.to_numeric(crsp['RET_x'], errors='coerce')
#Drop all NaN values in RET_x
crsp.dropna(subset=['RET_x'], inplace=True)

#### Loading the CRSP Index Data

In [20]:
crsp_index = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_index.csv")

In [21]:
crsp_index['caldt'] = pd.to_datetime(crsp_index['caldt'])
crsp_index.rename(columns={'caldt':'date'}, inplace=True)

In [22]:
crsp_index

,date,ewretd
0,1926-01-02,0.010
1,1926-01-04,0.006
2,1926-01-05,-0.002
3,1926-01-06,0.001
4,1926-01-07,0.008
...,...,...
23781,2015-12-24,0.002
23782,2015-12-28,-0.008
23783,2015-12-29,0.006
23784,2015-12-30,-0.007


In [ ]:
theta = 0.1
delta = 0.7
count=0
for permno in crsp['PERMNO'].unique():
    crsp_sample = crsp[crsp['PERMNO'] == permno].copy()
    crsp_sample = pd.merge(crsp_sample, crsp_index, on='date', how='left')
    crsp_sample['salience'] = abs(crsp_sample['RET_x'] - crsp_sample['ewretd']) / (
                abs(crsp_sample['ewretd']) + abs(crsp_sample['RET_x']) + theta)
    
    # Group by ticker and month and iterate over each group
    for name, group in crsp_sample.groupby(['TICKER', 'month']):
        # Rank the salience values
        group['salience_rank'] = group['salience'].rank(ascending=False)
        # Calculate the salience weight
        group['salience_weight'] = delta / (group['salience_rank'] * delta * (1 / len(group)))
        # Add the salience weight to the dataframe
        crsp_sample.loc[group.index, 'salience_weight'] = group['salience_weight']

        # Calculate Salience theory value ST
        cov_matrix = np.cov(group['RET_x'], group['salience_weight'])
        crsp_sample.loc[group.index, 'ST'] = cov_matrix[0][1]
    #Make the index of crsp_sample same as crsp['PERMNO'] == permno
    crsp_sample.set_index(crsp[crsp['PERMNO'] == permno].index, inplace=True)
    
    # Add the 'ST' column to the original DataFrame
    crsp.loc[crsp[crsp['PERMNO'] == permno].index, 'ST'] = crsp_sample['ST']
    count+=1
    if count%100 == 0:
        print("Processed PERMNO: ", count)

#### PLS SAVE ABOVE CRSP FOR NOT RUNNING ST AGAIN

In [ ]:
# crsp.to_csv("C:/Users/Aman/Downloads/Compressed/crsp_filtered_merged.csv")

### ONLY READ IF NOT RUNNING PREVIOUS CELLS -->

In [6]:
# crsp = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_filtered_merged.csv")
# crsp['date'] = pd.to_datetime(crsp['date'])

#### RUN THIS ONE NONE THE LESS (this is to add Shares outstanding) -->

In [8]:
crsp_monthly = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_us_equity_monthly_1.csv")
crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])
#Multiply SHROUT by 1000
crsp_monthly['SHROUT'] = crsp_monthly['SHROUT'] * 1000
crsp_monthly.drop(columns=['COMNAM','PRC','VOL','RET'], inplace=True)
# merge crsp and crsp_monthly on TICKER, PERMNO and date and drop a
crsp = pd.merge(crsp, crsp_monthly, on=['TICKER', 'PERMNO', 'date'], how='left')

In [12]:
# Create deciles based on the 'ST' column
crsp['Decile'] = pd.qcut(crsp['ST'], q=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                       labels=['Low', '2', '3', '4', '5', '6', '7', '8', '9', 'High'])
# Keep only month end dates and group by decile and take average of ST
crsp[crsp['date'].dt.is_month_end].groupby(['Decile'])['ST'].mean()*24

Decile
Low    -2.378
2      -1.090
3      -0.673
4      -0.363
5      -0.010
6       0.377
7       0.710
8       1.093
9       1.660
High    3.395
Name: ST, dtype: float64

In [13]:
crsp[crsp['date'].dt.is_month_end].groupby(['Decile'])['PRC_x'].mean()

Decile
Low    19.021
2      24.960
3      32.298
4      49.958
5      53.095
6      52.072
7      36.057
8      29.353
9      22.277
High   18.125
Name: PRC_x, dtype: float64

### ME Calculation

In [14]:
# Create a new columns called ME which is PRC * SHROUT_y
crsp['ME'] = crsp['PRC_x'] * crsp['SHROUT']

In [15]:
np.log(crsp[crsp['date'].dt.is_month_end].groupby(['Decile'])['ME'].mean())

Decile
Low    20.804
2      21.122
3      22.005
4      23.188
5      23.557
6      23.540
7      22.542
8      21.520
9      20.820
High   20.994
Name: ME, dtype: float64

#### BOOK VALUE OF EQUITY (BE) Calculation

In [16]:
# Read "C:\Users\Aman\Downloads\Compressed\compustat_us_equity.csv"
compustat_yearly = pd.read_csv("C:/Users/Aman/Downloads/Compressed/compustat_us_equity.csv")

In [17]:
# Drop indfmt	consol	popsrc	datafmt conm curcd costat columns
compustat_yearly.drop(columns=['indfmt', 'consol', 'popsrc', 'datafmt', 'conm', 'curcd', 'costat'], inplace=True)

In [18]:
#Rename datadate to date and convert it to datetime
compustat_yearly.rename(columns={'datadate':'date'}, inplace=True)
compustat_yearly['date'] = pd.to_datetime(compustat_yearly['date'])

#Rename bkvlps to book_value_per_share, csho to shares_outstanding, mkvalt to market_value
compustat_yearly.rename(columns={'bkvlps':'book_value_per_share', 'csho':'shares_outstanding', 'mkvalt':'market_value'}, inplace=True)

compustat_yearly['BE'] = compustat_yearly['book_value_per_share'] * compustat_yearly['shares_outstanding'] * 1000000

In [19]:
compustat_yearly

,gvkey,date,fyear,tic,book_value_per_share,shares_outstanding,market_value,BE
0,1000,1961-12-31,1961.000,AE.2,2.434,0.152,NaN,369998.400
1,1000,1962-12-31,1962.000,AE.2,3.050,0.181,NaN,551995.700
2,1000,1963-12-31,1963.000,AE.2,2.973,0.186,NaN,552996.600
3,1000,1964-12-31,1964.000,AE.2,3.097,0.196,NaN,606992.400
4,1000,1965-12-31,1965.000,AE.2,2.384,0.206,NaN,491001.000
...,...,...,...,...,...,...,...,...
515983,328795,2013-12-31,2013.000,ACA,NaN,NaN,NaN,NaN
515984,328795,2014-12-31,2014.000,ACA,NaN,NaN,NaN,NaN
515985,328795,2015-12-31,2015.000,ACA,NaN,NaN,NaN,NaN
515986,335466,2015-12-31,2015.000,HOFSQ,NaN,NaN,NaN,NaN


#### Loading the Key to Connect CRSP and Compustat (Permno to GVKEY)

In [20]:
permno_gvkey = pd.read_csv("C:/Users/Aman/Downloads/Compressed/permno_gvkey.csv")

In [21]:
permno_gvkey["LINKDT"] = pd.to_datetime(permno_gvkey["LINKDT"])

# Replace "E" in LINKENDDT with today's date
permno_gvkey["LINKENDDT"].replace({"E": '2016-01-01'}, inplace=True)

permno_gvkey["LINKENDDT"] = pd.to_datetime(permno_gvkey["LINKENDDT"])

permno_gvkey

,GVKEY,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,CONM
0,1000,LU,25881,23369,1970-11-13,1978-06-30,A & E PLASTIK PAK INC
1,1001,LU,10015,6398,1983-09-20,1986-07-31,A & M FOOD SERVICES INC
2,1002,LC,10023,22159,1972-12-14,1973-06-05,AAI CORP
3,1003,LU,10031,6672,1983-12-07,1989-08-16,A.A. IMPORTING CO INC
4,1004,LU,54594,20000,1972-04-24,2016-01-01,AAR CORP
...,...,...,...,...,...,...,...
32928,349994,LC,23514,59438,2022-11-15,2016-01-01,CLEARMIND MEDICINE INC
32929,350681,LC,22205,58855,2021-10-22,2023-03-31,GETNET ADQUIRENCIA E
32930,351038,LC,16161,55612,2021-10-29,2016-01-01,QUOIN PHARMACEUTICALS LTD
32931,352262,LC,23773,59507,2023-03-17,2016-01-01,COOL COMPANY LTD


In [22]:
# Create a dictionary with GVKEY as key and PERMNO as value.
permno_gvkey_dict = dict(zip(permno_gvkey['GVKEY'], permno_gvkey['LPERMNO']))

In [23]:
# Use the disctionary permno_gvkey_dict to add a new column PERMNO to compustat_yearly
compustat_yearly['PERMNO'] = compustat_yearly['gvkey'].map(permno_gvkey_dict)

In [24]:
#Count the number of missing PERMNO values
compustat_yearly['PERMNO'].isna().sum()
# Drop all rows with missing PERMNO values
compustat_yearly.dropna(subset=['PERMNO'], inplace=True)
# Convert PERMNO to int
compustat_yearly['PERMNO'] = compustat_yearly['PERMNO'].astype(int)
compustat_yearly

,gvkey,date,fyear,tic,book_value_per_share,shares_outstanding,market_value,BE,PERMNO
0,1000,1961-12-31,1961.000,AE.2,2.434,0.152,NaN,369998.400,25881
1,1000,1962-12-31,1962.000,AE.2,3.050,0.181,NaN,551995.700,25881
2,1000,1963-12-31,1963.000,AE.2,2.973,0.186,NaN,552996.600,25881
3,1000,1964-12-31,1964.000,AE.2,3.097,0.196,NaN,606992.400,25881
4,1000,1965-12-31,1965.000,AE.2,2.384,0.206,NaN,491001.000,25881
...,...,...,...,...,...,...,...,...,...
515983,328795,2013-12-31,2013.000,ACA,NaN,NaN,NaN,NaN,18144
515984,328795,2014-12-31,2014.000,ACA,NaN,NaN,NaN,NaN,18144
515985,328795,2015-12-31,2015.000,ACA,NaN,NaN,NaN,NaN,18144
515986,335466,2015-12-31,2015.000,HOFSQ,NaN,NaN,NaN,NaN,14756


In [25]:
# Read table from text file DFF_BE_With_Nonindust.txt in same folder with sep as space
equity_old = pd.read_csv("C:/Users/Aman/Downloads/Compressed/kenneth_old_data.csv")

In [26]:
equity_old.drop(columns=['first_moody','last_moody'], inplace=True)
equity_old_melted = pd.melt(equity_old, id_vars=['PERMNO'], var_name='date', value_name='value')

# Conver the year to end of year date
equity_old_melted['date'] = pd.to_datetime(equity_old_melted['date'], format='%Y')
equity_old_melted['date'] = equity_old_melted['date'] + pd.offsets.YearEnd(0)

equity_old_melted['BE'] = equity_old_melted['value'] * 1000000
equity_old_melted.drop(columns=['value'], inplace=True)
equity_old_melted

,PERMNO,date,BE
0,10006,1926-12-31,67743000.000
1,10014,1926-12-31,13005000.000
2,10022,1926-12-31,13567000.000
3,10030,1926-12-31,15924000.000
4,10049,1926-12-31,11984000.000
...,...,...,...
136339,86134,2001-12-31,-99990000.000
136340,86239,2001-12-31,-99990000.000
136341,86861,2001-12-31,-99990000.000
136342,92567,2001-12-31,-99990000.000


In [27]:
# Merge compustat_yearly and equity_old_melted on PERMNO and date
compustat_yearly = pd.merge(compustat_yearly, equity_old_melted, on=['PERMNO', 'date'], how='outer')

In [28]:
#Create a combined book_value_per_share column
compustat_yearly['BE'] = compustat_yearly['BE_x'].fillna(compustat_yearly['BE_y'])

In [29]:
compustat_yearly.drop(columns=['BE_x', 'BE_y'], inplace=True)

In [30]:
#Drop all rows with missing book_value_per_share values
compustat_yearly.dropna(subset=['book_value_per_share'], inplace=True)
#Drop all rows with negative book_value_per_share values
compustat_yearly = compustat_yearly[compustat_yearly['book_value_per_share'] > 0]

In [31]:
compustat_yearly_for_merge = compustat_yearly.copy()
compustat_yearly_for_merge.drop(columns=['gvkey', 'fyear','tic','shares_outstanding','market_value','book_value_per_share'], inplace=True)
compustat_yearly_for_merge

,date,PERMNO,BE
0,1961-12-31,25881,369998.400
1,1962-12-31,25881,551995.700
2,1963-12-31,25881,552996.600
3,1964-12-31,25881,606992.400
4,1965-12-31,25881,491001.000
...,...,...,...
413580,2015-12-31,16496,5361991681.200
413581,2013-12-31,15904,1260040.000
413582,2014-12-31,15904,66095521.700
413583,2015-12-31,15904,67665602.400


In [32]:
# add the book_value_per_share to crsp
crsp = pd.merge(crsp, compustat_yearly_for_merge, on=['PERMNO', 'date'], how='left')
crsp

,PERMNO,date,TICKER,VOL_x,RET_x,month,days_in_month,PRC,VOL_y,RET_y,PRC_x,ST,SHROUT,Decile,ME,BE
0,10495,1962-07-02,A,2600.000,0.022,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN
1,10495,1962-07-03,A,2100.000,0.006,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN
2,10495,1962-07-05,A,3600.000,-0.003,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN
3,10495,1962-07-06,A,2600.000,-0.018,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN
4,10495,1962-07-09,A,4000.000,0.006,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59173811,91205,2008-10-27,ZZ,443900.000,-0.029,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN
59173812,91205,2008-10-28,ZZ,369000.000,0.015,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN
59173813,91205,2008-10-29,ZZ,610400.000,0.091,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN
59173814,91205,2008-10-30,ZZ,718200.000,0.100,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN


In [33]:
# Calculate book to market ratio
crsp['BM'] = crsp['BE'] / crsp['ME']
crsp

,PERMNO,date,TICKER,VOL_x,RET_x,month,days_in_month,PRC,VOL_y,RET_y,PRC_x,ST,SHROUT,Decile,ME,BE,BM
0,10495,1962-07-02,A,2600.000,0.022,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN,NaN
1,10495,1962-07-03,A,2100.000,0.006,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN,NaN
2,10495,1962-07-05,A,3600.000,-0.003,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN,NaN
3,10495,1962-07-06,A,2600.000,-0.018,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN,NaN
4,10495,1962-07-09,A,4000.000,0.006,1962-07,21.000,NaN,NaN,NaN,40.375,-0.019,NaN,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59173811,91205,2008-10-27,ZZ,443900.000,-0.029,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN,NaN
59173812,91205,2008-10-28,ZZ,369000.000,0.015,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN,NaN
59173813,91205,2008-10-29,ZZ,610400.000,0.091,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN,NaN
59173814,91205,2008-10-30,ZZ,718200.000,0.100,2008-10,23.000,NaN,NaN,NaN,6.460,-0.147,NaN,Low,NaN,NaN,NaN


In [34]:
np.log(crsp[crsp['date'].dt.is_year_end].groupby(['Decile'])['BM'].mean())

Decile
Low    0.711
2      1.044
3      1.313
4      1.014
5      1.025
6      1.059
7      1.114
8      1.757
9      1.600
High   0.620
Name: BM, dtype: float64

In [35]:
# Count how many 'C' values are there in the 'RET_y' column
crsp[crsp['RET_y']=='C']
# Take all the rows where 'RET_y' is 'C' and compute it using the RET_x cumulatively
crsp.loc[crsp[crsp['RET_y']=='C'].index, 'RET_y'] = crsp.loc[crsp[crsp['RET_y']=='C'].index, 'RET_x'].cumsum()

In [65]:
#save crsp_monthly
crsp_monthly = crsp[crsp['date'].dt.is_month_end]

In [67]:
# cum_monthly_ret is the 13 month cumulative return - 2 month cumulative return
crsp_monthly['cum_monthly_ret'] = crsp_monthly.groupby(['PERMNO'])['RET_y'].transform(lambda x: x.rolling(13).sum() - x.rolling(2).sum())

In [70]:
# rename cum_monthly_ret to MOM
crsp_monthly.rename(columns={'cum_monthly_ret':'MOM'}, inplace=True)

In [71]:
crsp_monthly.groupby(['Decile'])['MOM'].mean()*100

Decile
Low    20.581
2      19.490
3      19.009
4      18.140
5      16.427
6      18.492
7      20.468
8      21.638
9      21.503
High   20.115
Name: MOM, dtype: float64

#### ILLIQ

In [76]:
# Load the daily price data
crsp_price = pd.read_csv("C:/Users/Aman/Downloads/Compressed/crsp_us_equity.csv")

In [78]:
crsp_price.drop(columns=['TICKER','VOL','RET'], inplace=True)

In [79]:
crsp_price['date'] = pd.to_datetime(crsp_price['date'])

In [80]:
# merge crsp_price into crsp
crsp = pd.merge(crsp, crsp_price, on=['date', 'PERMNO'], how='left')

In [121]:
crsp.drop(columns=['PRC_x'], inplace=True)

In [127]:
crsp['ILLIQ'] = abs(crsp['RET_x']) *1000000 / (crsp['VOL_x'] * crsp['PRC_y'])

In [137]:
# find inf values in ILLIQ and replace them with NaN
crsp['ILLIQ'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [141]:
#Create a new columns 'ILLIQ_monthly' which is the average of ILLIQ for each month
crsp['ILLIQ_monthly'] = crsp.groupby(['PERMNO', 'month'])['ILLIQ'].transform('mean')

In [143]:
crsp

,PERMNO,date,TICKER,VOL_x,RET_x,month,days_in_month,VOL_y,RET_y,ST,SHROUT,Decile,ME,BE,BM,cum_monthly_ret,PRC_y,ILLIQ,ILLIQ_monthly
0,10495,1962-07-02,A,2600.000,0.022,1962-07,21.000,NaN,NaN,-0.019,NaN,4,NaN,NaN,NaN,NaN,41.125,0.203,0.057
1,10495,1962-07-03,A,2100.000,0.006,1962-07,21.000,NaN,NaN,-0.019,NaN,4,NaN,NaN,NaN,NaN,41.375,0.070,0.057
2,10495,1962-07-05,A,3600.000,-0.003,1962-07,21.000,NaN,NaN,-0.019,NaN,4,NaN,NaN,NaN,NaN,41.250,0.020,0.057
3,10495,1962-07-06,A,2600.000,-0.018,1962-07,21.000,NaN,NaN,-0.019,NaN,4,NaN,NaN,NaN,NaN,40.500,0.173,0.057
4,10495,1962-07-09,A,4000.000,0.006,1962-07,21.000,NaN,NaN,-0.019,NaN,4,NaN,NaN,NaN,NaN,40.750,0.038,0.057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59173811,91205,2008-10-27,ZZ,443900.000,-0.029,2008-10,23.000,NaN,NaN,-0.147,NaN,Low,NaN,NaN,NaN,NaN,2.710,0.024,0.032
59173812,91205,2008-10-28,ZZ,369000.000,0.015,2008-10,23.000,NaN,NaN,-0.147,NaN,Low,NaN,NaN,NaN,NaN,2.750,0.015,0.032
59173813,91205,2008-10-29,ZZ,610400.000,0.091,2008-10,23.000,NaN,NaN,-0.147,NaN,Low,NaN,NaN,NaN,NaN,3.000,0.050,0.032
59173814,91205,2008-10-30,ZZ,718200.000,0.100,2008-10,23.000,NaN,NaN,-0.147,NaN,Low,NaN,NaN,NaN,NaN,3.300,0.042,0.032


In [105]:
# count=0
# for permno in crsp['PERMNO'].unique()[:100]:
#     crsp_sample = crsp[crsp['PERMNO'] == permno].copy()
    
#     # Group by ticker and month and iterate over each group
#     for name, group in crsp_sample.groupby(['TICKER', 'month']):
#         # illiquidity (ILLIQ) is computed as the absolute daily return divided by the daily dollar trading volume, averaged
#         # over all trading days in a month
#         group['ILLIQ'] = abs(group['RET_x']) / (group['PRC_y'] * group['VOL_x'])
        
#         # Add the salience weight to the dataframe
#         crsp_sample.loc[group.index, 'ILLIQ'] = group['ILLIQ']

#         # Calculate Salience theory value ST
#         illiq_mean = group['ILLIQ'].mean()
#         crsp_sample.loc[group.index, 'ILLIQ'] = illiq_mean * 1000000
#     #Make the index of crsp_sample same as crsp['PERMNO'] == permno
#     crsp_sample.set_index(crsp[crsp['PERMNO'] == permno].index, inplace=True)
    
#     # Add the 'ILLIQ' column to the original DataFrame
#     crsp.loc[crsp[crsp['PERMNO'] == permno].index, 'ILLIQ'] = crsp_sample['ILLIQ']
#     count+=1
#     if count%100 == 0:
#         print("Processed PERMNO: ", count)

Processed PERMNO:  100


In [146]:
crsp.groupby(['Decile'])['ILLIQ'].mean()

Decile
Low    0.903
2      0.541
3      0.383
4      0.235
5      0.105
6      0.225
7      0.346
8      0.436
9      0.595
High   0.675
Name: ILLIQ, dtype: float64

In [145]:
crsp.groupby(['Decile'])['ILLIQ_monthly'].mean()

Decile
Low    1.192
2      0.757
3      0.525
4      0.323
5      0.135
6      0.304
7      0.459
8      0.595
9      0.807
High   0.989
Name: ILLIQ_monthly, dtype: float64

In [144]:
# show decile wise average ILLIQ
crsp[crsp['date'].dt.is_month_end].groupby(['Decile'])['ILLIQ_monthly'].mean()

Decile
Low    1.030
2      0.756
3      0.513
4      0.326
5      0.130
6      0.303
7      0.457
8      0.589
9      0.819
High   0.977
Name: ILLIQ_monthly, dtype: float64

### 3. 
From Tables 3-10, choose two other tables and replicate them.

### 4. 
If the numbers you obtain in questions 2 and 3 deviate from those in the paper, why do you think this is? What parts of the data construction and replication were difficult? Was there any additional information the authors could have given you to make this process simpler?

- We removed the stocks from scope of the daily CRSP dataset if the Monthly Close Price (or Bid-Ask Avg. depending on data availability) was less than or equal to $5. This was done to mitigate market microstructure effects.

### 5. 
In your view, what are the key takeaways of this paper? How did the results in the tables you replicated contribute to the paper as a whole?